In [6]:
import numpy as np
from tqdm import tqdm
from time import time
import json, pickle, os, string, tqdm, kenlm, json
from collections import defaultdict, Counter
from itertools import groupby
import Levenshtein as Lev
import math


In [7]:
#s1 = True text
#s2 = predicted text

def wer_(s1, s2):
    """
    Computes the Word Error Rate, defined as the edit distance between the
    two provided sentences after tokenizing to words.
    Arguments:
        s1 (string): space-separated sentence
        s2 (string): space-separated sentence
    """

    # build mapping of words to integers
    b = set(s1.split() + s2.split())
    word2char = dict(zip(b, range(len(b))))

    # map the words to a char array (Levenshtein packages only accepts
    # strings)
    w1 = [chr(word2char[w]) for w in s1.split()]
    w2 = [chr(word2char[w]) for w in s2.split()]
    
    return Lev.distance(''.join(w1), ''.join(w2))

def cer_(s1, s2):
    """
    Computes the Character Error Rate, defined as the edit distance.

    Arguments:
        s1 (string): space-separated sentence
        s2 (string): space-separated sentence
    """
    s1, s2, = s1.replace(' ', ''), s2.replace(' ', '')

    return Lev.distance(s1, s2)




#When using the above implementation, use the code belove to calculate the wer in percentatge: 
#pred = list of ouput prediction of model (it is the text) # example [" MY NAME IS HEMANT", " I AM A GOD"]
# total_wer = 0
# for x in range(len(pred)):
#     transcript, reference = data_[x][1], pred[x]
#     wer_inst = wer(transcript, reference)
#     total_wer += float(wer_inst)
# print("WER is : ",total_wer/len(pred),"%")
# wer_(pred,true)/len(pred.split(' '))*100

In [8]:
def ctc_best_path(out,labels):
    "implements best path decoding as shown by Graves"
    out = [labels[i] for i in np.argmax(out, axis=1) if i!=labels[-1]]
    o = ""
    for i,j in groupby(out):
        o = o + i
    return o.replace("_","")

In [9]:
gred_txt = ctc_best_path(out,labels)
print(gred_txt)
wer_(gred_txt,reference)/len(gred_txt.split(' '))*100

NameError: name 'out' is not defined

### WORD LM Implementation

In [128]:
def lse(*args):
    a_max = max(args)
    lsp = math.log(sum(math.exp(a-a_max) for a in args))
    return a_max + lsp

In [130]:
lse(1,2,3)

3.4076059644443806

In [135]:
math.log(sum(math.exp(a) for a in [1,2,3]))

3.40760596444438

In [97]:
lm_w = kenlm.LanguageModel('/home/hemant/sopi_deep/lm/3_gram_full.arpa')

In [121]:
def sort_beam(ptot,k):
    if len(ptot) < k:
        return [i for i in ptot.keys()]
    else:
        dict_ = sorted(dict((v,k) for k,v in ptot.items()).items(),reverse=True)[:k]

        return [i[1] for i in dict_]

#using WORD LM
def ctc_beam_search(out,labels, prune=0.0001, k=20, lm=None,alpha=0,beta=0):
    "implements CTC Prefix Search Decoding Algorithm as shown by Graves"
    '''
    out = ctc output
    labels = string of labels
    prune = prune the ctc output
    k=beam-width
    lm=word age model used
    alpha,beta = hyper-parameters
    '''

    bc_i = 0 # blank/special charatcter index 
    F = out.shape[1]
    out = np.vstack((np.zeros(F), out))
    steps = out.shape[0]

    pb, pnb = defaultdict(Counter), defaultdict(Counter)
    pb[0][''], pnb[0][''] = 1, 0
    prev_beams = ['']
    for t in range(1,steps):
        pruned_alphabet = [labels[i] for i in np.where(out[t] > prune)[0]]
        for b in prev_beams:
            for c_t in pruned_alphabet:
                index = labels.index(c_t)
                #Collapsing case (copy case as the last character in the beam)
                if c_t == "_": #Extending with a blank
                    pb[t][b] += out[t][index]*(pb[t-1][b] + pnb[t-1][b])
                    continue
                else:
                    i_plus = b + c_t
                    if len(b) > 0 and c_t == b[-1]: #Extending with the same character as the last one
                        pnb[t][b] += out[t][index]*pnb[t-1][b]
                        pnb[t][i_plus] += out[t][index]*pb[t-1][b]
                    #expanding the beam (extend case as the last character is different)
                    elif c_t == " " and len(b.replace(' ', '')) > 0 : # LM constraints
                        prob = [i[0] for i in lm.full_scores(' '.join(i_plus.split(" ")[-4:-1]),eos=False,bos=False)][-1]
                        lm_p = (10**prob)**alpha
                        pnb[t][i_plus] += lm_p*out[t][index]*(pb[t-1][b] + pnb[t-1][b])
                    else:
                        pnb[t][i_plus] += out[t][index]*(pb[t-1][b] + pnb[t-1][b])

                    if i_plus not in prev_beams:
                        pb[t][i_plus] += out[t][index] * (pb[t - 1][i_plus] + pnb[t - 1][i_plus])
                        pnb[t][i_plus] += out[t][index] * pnb[t - 1][i_plus]
        
        ptot = pb[t] + pnb[t]
        for i in ptot.keys():
            ptot[i] = (ptot[i] * (len(i.split(' '))+1))**beta
        prev_beams= sort_beam(ptot,k)
    return prev_beams[0]

In [127]:
beam_txt=ctc_beam_search(out,labels,0.00001,k=100,lm=lm_w,alpha=0,beta=6)
print(beam_txt)
wer_(beam_txt,reference)/len(beam_txt.split(' '))*100

I THINK IS IS GOINGNA BE VORY ALD IDEA IN THE BEAST IN INTELLETION ALDATI BUSINESSES IN THE YEARS TO COME I DONTWT


52.17391304347826

In [126]:
reference

'I THINK ITS ITS GONNA BE VERY GOOD IDEA IN THE BIS IN THE INTERNATIONAL BISHNES BUSINESSES IN THE YEARS TO COME XXXAH I DONT HAVE'

### CHARACTER LM Implementation


In [14]:
lm_c = kenlm.LanguageModel('/home/hemant/junk/3_gram.arpa')

In [53]:
def sort_beam(ptot,k):
    if len(ptot) < k:
        return [i for i in ptot.keys()]
    else:
        dict_ = sorted(dict((v,k) for k,v in ptot.items()).items(),reverse=True)[:k]
        return [i[1] for i in dict_]

#using CHARACTER LM
def ctc_beam_search_clm(out,labels, prune=0.001, k=20, lm_w = None, lm_c=None,alpha=0,beta=0):
    "implements CTC Prefix Search Decoding Algorithm as shown by Graves"
    
    '''
    out = ctc output
    labels = string of labels
    prune = prune the ctc output
    k=beam-width
    lm=charac language model used
    alpha,beta = hyper-parameters
    '''
    
    bc_i = 0 # special charatcter index 
    F = out.shape[1]
    out = np.vstack((np.zeros(F), out))
    steps = out.shape[0]
    
    pb, pnb = defaultdict(Counter), defaultdict(Counter)
    pb[0][''], pnb[0][''] = 1, 0
    prev_beams = ['']
    for t in range(1,steps):
        pruned_alphabet = [labels[i] for i in np.where(out[t] > prune)[0]]
        for b in prev_beams:
            for c_t in pruned_alphabet:
                index = labels.index(c_t)
                #Collapsing case (copy case as the last character in the beam)
                if c_t == "_": #Extending with a blank
                    pb[t][b] += out[t][index]*(pb[t-1][b] + pnb[t-1][b])  
                else:  # LM constraints
                    i_plus = b + c_t
                     #Extending with the same character as the last one
                    if len(b) > 0 and c_t == b[-1]:
                        pnb[t][b] += out[t][index]*pnb[t-1][b]
                        pnb[t][i_plus] += out[t][index]*pb[t-1][b]
                        
                    #expanding the beam (extend case as the last character is different)
                    elif len(b.replace(' ', '')) > 0 :                       
#                         if c_t == " " and lm_w != None: # word LM constraints
#                             if lm_w != None:
#                                 prob = [i[0] for i in lm_w.full_scores(' '.join(i_plus.split(" ")[-4:-1]),eos=False,bos=False)][-1]
#                                 lm_p = (10**prob)**alpha
#                             else: lm_p =1
#                             pnb[t][i_plus] += lm_p*out[t][index]*(pb[t-1][b] + pnb[t-1][b])
#                         else: # character LM constraint
                        if lm_c != None:
                            prob = [i[0] for i in lm_c.full_scores(i_plus,eos=False,bos=False)][-1]
                            lm_p = (10**prob)**alpha
                        else: lm_p = 1
                        pnb[t][i_plus] += lm_p*out[t][index]*(pb[t-1][b] + pnb[t-1][b])
                    
                    else:
                        pnb[t][i_plus] += out[t][index]*(pb[t-1][b] + pnb[t-1][b])
                        
                    if i_plus not in prev_beams:
                        pb[t][i_plus] += out[t][index] * (pb[t - 1][i_plus] + pnb[t - 1][i_plus])
                        pnb[t][i_plus] += out[t][index] * pnb[t - 1][i_plus]
                        
        ptot = pb[t] + pnb[t]
        for i in ptot.keys():
            ptot[i] = ptot[i]*(len(i.split())+1)**beta
        prev_beams = sort_beam(ptot,k)
    return prev_beams[0]

# IMPLEMANTATION

In [62]:
import os
os.chdir("/home/hemant/E2E_NER-Through-Speech/S2T/")
from opts import add_decoder_args, add_inference_args
from utils import load_model
import os
from ctc_decoders import *
import argparse
import numpy as np
import torch
from tqdm import tqdm
from data.data_loader import SpectrogramParser

from opts import add_decoder_args, add_inference_args
from utils import load_model

In [63]:
labels = "_'ABCDEFGHIJKLMNOPQRSTUVWXYZ "

In [85]:
# decoding = 'greedy'
decoding = 'beam_w'
# decoding = 'beam_c'
prune = 0.00001
beam_width = 10
alpha = 10
beta = 24
lm = lm_w

torch.set_grad_enabled(False)
device = torch.device("cuda")
model = load_model(device, "/home/hemant/sopi_deep/models/deep/finetuen_sopi.pth")
spect_parser = SpectrogramParser(model.audio_conf, normalize=True)

torch.cuda.set_device(int(0))
with open("/home/hemant/sopi_deep/data/deepspeech/test/ata/test.csv","r") as f:
    csv = f.readlines()

total_cer, total_wer, num_tokens, num_chars = 0, 0, 0, 0
lm = lm_w
output = []
for i in tqdm(csv):
    audio_path, reference_path = i.split(",")

    spect = spect_parser.parse_audio(audio_path).contiguous()
    spect = spect.view(1, 1, spect.size(0), spect.size(1))
    spect = spect.to(device)

    input_sizes = torch.IntTensor([spect.size(3)]).int()
    out, output_sizes = model(spect, input_sizes)
    out = out.cpu().detach().numpy()[0]

    if decoding == "greedy": transcript = ctc_best_path(out,labels)
    elif decoding == "beam_w": transcript = ctc_beam_search(out,labels,prune,beam_width,lm,alpha,beta)
    elif decoding == "beam_c": transcript = ctc_beam_search_clm(out,labels,prune,beam_width,lm,alpha=alpha,beta=beta)
        
    with open(reference_path.replace("\n",""),"r") as f:
        reference = f.readline()
    output.append([transcript,reference])
    wer_inst = wer_(transcript,reference)
    cer_inst = cer_(transcript, reference)
    total_wer += wer_inst
    total_cer += cer_inst
    num_tokens += len(reference.split())
    num_chars += len(reference.replace(' ', ''))
        
wer = (float(total_wer) / num_tokens)*100
cer = (float(total_cer) / num_chars)*100
print('Test Summary \t'
    'Average WER {wer:.3f}\t'
    'Average CER {cer:.3f}\t'.format(wer=wer, cer=cer))



  0%|          | 0/50 [00:00<?, ?it/s]

  2%|▏         | 1/50 [00:00<00:17,  2.86it/s]

  4%|▍         | 2/50 [00:00<00:16,  2.98it/s]

  6%|▌         | 3/50 [00:01<00:17,  2.69it/s]

  8%|▊         | 4/50 [00:01<00:18,  2.54it/s]

 10%|█         | 5/50 [00:02<00:18,  2.37it/s]

 12%|█▏        | 6/50 [00:02<00:14,  2.94it/s]

 14%|█▍        | 7/50 [00:02<00:14,  3.02it/s]

 16%|█▌        | 8/50 [00:02<00:14,  2.90it/s]

 18%|█▊        | 9/50 [00:03<00:13,  2.97it/s]

 20%|██        | 10/50 [00:03<00:12,  3.10it/s]

 22%|██▏       | 11/50 [00:03<00:12,  3.01it/s]

 24%|██▍       | 12/50 [00:04<00:13,  2.88it/s]

 26%|██▌       | 13/50 [00:04<00:12,  3.00it/s]

 28%|██▊       | 14/50 [00:04<00:11,  3.18it/s]

 30%|███       | 15/50 [00:05<00:11,  3.14it/s]

 32%|███▏      | 16/50 [00:05<00:11,  3.04it/s]

 34%|███▍      | 17/50 [00:05<00:11,  2.95it/s]

 36%|███▌      | 18/50 [00:06<00:11,  2.80it/s]

 38%|███▊      | 19/50 [00:06<00:10,  2.92it/s]

 40%|████      | 20/50 [00:06<00:10,

Test Summary 	Average WER 99.764	Average CER 25.093	


In [86]:
for i in output:
    print(i,'\n')

[' OURPRODATIVITYANDOURCOMMITMENTTOOURACLIENTSXXXUMTOMAKESURETHATWENEEDWHATBEXXXUMPROMISED FORTHE DEADLINE', 'OUR PRODUCTIVITY AND OUR COMMITMENT TO OUR CLIENTS XXXUM TO MAKE SURE WHAT WE NEED WHAT WE PROMISED FOR THE DEADLINE'] 

['    MEALMYHISIMPORTANTXXXUMIIWOULDTELLMSURMURPHYTHATXXXUMTHOUHWEWOULDPAYBU', 'SO IT REALLY IS IMPORTANT I WOULD TELL MR MURPHY THAT THOUGH WE WOULD PAY'] 

[' HIMISWILSONTHISISRUNXXXUMYMACHLYGONBECAUSEXXXUHIMSTILLANTHEIRFORTHEFLIGHTSANDDELAYESINCETHEREISBADSTORM', 'HI MISS WILSON THIS IS RYAN XXXUM IM ACTUALLY CALLING BECAUSE XXXUH IM STILL IN AIRPORT THE FLIGHTS BEEN DELAYED SINCE THERE IS A BAD STORM'] 

[' SOIIUCANNOTICANATTRAINTAMEETINGIMSORRYIIRIGHIREALLYPOTRIIESFORWIRTHISSOMESSAGE', 'SO I CANNOT I CANNOT JOIN THE MEETING I XXXST I REALLY APOLOGISE FOR THIS MESSAGE'] 

['  YOUKNOWITSNOTWHATIAHADPLANNEDXHYOHICASXXHIEJUSTTEVERIKEEPYOUNOTD', 'BAD BUT YOU KNOW ITS NOT WHAT I HAD PLANNED SO YEAH I GUESS ILL I JUST BE FOR KEEP YOU NOTIFIED'] 

['  THEYAREONT

In [87]:
transcript

'    THINKISISGOINGNABEAVORYCALDAIDEAINTHEBEASTINTINTELLETIONALDATIBUSINESSESINTHERYEARSTOCOMEIDONTWT'

In [88]:
reference

'I THINK ITS ITS GONNA BE VERY GOOD IDEA IN THE BIS IN THE INTERNATIONAL BISHNES BUSINESSES IN THE YEARS TO COME XXXAH I DONT HAVE'

In [ ]:
a = [9,33,46,50,31,50,0,37,14,0,53,28,20,0,18,46,0,100,25,30,71,20,4,0,3,21,3,15,22,3,0,27,35,6,56,95,46,50,35,20,33,0,25,19,18,50,1100,8,0,30]

In [ ]:
sum(a)

In [ ]:
sum(a)/50

In [ ]:
print(j)

In [ ]:
m = audio_path + "," +reference_path
with open("/home/hemant/junk/out.csv","w") as f:
    f.write(m)

In [ ]:
transcript

In [ ]:
reference

In [ ]:
wer_(transcript,reference)/len(transcript.split(' '))*100

In [ ]:
gred_txt = ctc_best_path(out,labels)
print(gred_txt)
wer_(gred_txt,reference)/len(gred_txt.split(' '))*100

In [ ]:
m

In [ ]:
beam_txt=ctc_beam_search(out,labels,0.0001,k=100,lm=lm_w,alpha=0.3,beta=12)
print(beam_txt)
wer_(beam_txt,reference)/len(beam_txt.split(' '))*100

In [ ]:
beam_txt=ctc_beam_search_clm(out,labels,0,k=10, lm_c = lm_c, alpha=0.1,beta=6)
print(beam_txt)
wer_(beam_txt,reference)/len(beam_txt.split(' '))*100